In [3]:
# public key generation
import util
from test_framework.key import ECKey, ECPubKey, generate_key_pair, generate_bip340_key_pair, generate_schnorr_nonce
from test_framework.messages import sha256
from test_framework.musig import aggregate_musig_signatures, aggregate_schnorr_nonces, generate_musig_key, sign_musig
from test_framework.script import tagged_hash

priv1, pub1=generate_key_pair(sha256(b'key0'))
priv2, pub2=generate_key_pair(sha256(b'key1'))
priv3, pub3=generate_key_pair(sha256(b'key2'))
pubkeys = [pub1,pub2,pub3]
c_map, pubkey_agg=generate_musig_key(pubkeys)

priv1_c=priv1*c_map.get(pub1)
pub1_c=pub1*c_map.get(pub1)
priv2_c=priv2*c_map.get(pub2)
pub2_c=pub2*c_map.get(pub2)
priv3_c=priv3*c_map.get(pub3)
pub3_c=pub3*c_map.get(pub3)
if pubkey_agg.get_y() %2 !=0 :
    priv1_c.negate()
    pub1_c.negate()
    priv2_c.negate()
    pub2_c.negate()
    priv3_c.negate()
    pub3_c.negate()
assert priv1_c.secret == 104717570570407299858230629579807834166658508605015363884161538594382975780625
assert priv2_c.secret == 65554880484297966965546994775376394861215085064604177497808278620612854069980
assert priv3_c.secret == 106998690642216524894360365246223287721822845133760006050846956016514597569168

In [10]:
# generate nonce and nonce point
k1=ECKey().set(101)
k2=ECKey().set(222)
k3=ECKey().set(333)
test_k1=ECKey().set(k1.secret)
test_k2=ECKey().set(k2.secret)
test_k3=ECKey().set(k3.secret)

# get nonce point
R1=test_k1.get_pubkey()
R2=test_k2.get_pubkey()
R3=test_k3.get_pubkey()

# generate nonce point and commitments and exchange them
R1_digest = sha256(R1.get_bytes())
R2_digest = sha256(R2.get_bytes())
R3_digest = sha256(R3.get_bytes())

assert R1_digest.hex() == "38018cfa00483e751b166e7d982a5bb8264fb3309739c2f432e79791a1c9aaf7"
assert R2_digest.hex() == "9eb8fac583a9d83d4753c454e4ab4de833b3496d093a6f2df507a6a39424c745"
assert R3_digest.hex() == "103ea7eeb151bc6bd2c1e54ecaaad303b1c022bb205c5430daac796924a80ed0"

R_agg = R1 + R2 + R3
if R_agg.get_y() %2 != 0:
    R_agg.negate()
    k1.negate()
    k2.negate()
    k3.negate()

test_R_agg, negated= aggregate_schnorr_nonces([R1,R2,R3])
if negated:
    test_k1.negate()
    test_k2.negate()
    test_k3.negate()

assert R_agg == test_R_agg
assert k1 == test_k1
assert k2 == test_k2
assert k3 == test_k3
msg = sha256(b'transaction')
s1=sign_musig(priv1_c,k1,R_agg,pubkey_agg,msg)
s2=sign_musig(priv2_c,k2,R_agg,pubkey_agg,msg)
s3=sign_musig(priv3_c,k3,R_agg,pubkey_agg,msg)
sig_agg = aggregate_musig_signatures([s1,s2,s3],R_agg)
assert pubkey_agg.verify_schnorr(sig_agg,msg)

TypeError: object of type 'int' has no len()